In [5]:
import sys
sys.path.append('../')

import os
import pandas as pd
import common
from shutil import copyfile
import one_part_dataset

In [6]:
metadata = common.load_metadata_dataframe('state.json')

In [8]:
metadata[metadata["useful"] == "yes"].shape

(4738, 4)

In [7]:
FROM_PREFIX = '/Users/fcuevas/Documents/fing/tesis/datos/FOTOS SINIESTROS/EMPRESA CARS/FOTOS DENUNCIA/{}'
TO_PREFIX = "./imgs_final/{}"

def copy_img(row):
    from_dir = FROM_PREFIX.format(row["image"])
    to_dir = TO_PREFIX.format(row["image"])
    
    os.makedirs(os.path.dirname(to_dir), exist_ok=True)
    copyfile(from_dir, to_dir)

metadata.apply(copy_img, axis=1)

0       None
1       None
2       None
3       None
4       None
        ... 
4738    None
4739    None
4740    None
4741    None
4742    None
Length: 4738, dtype: object

In [4]:
metadata[metadata["useful"] == "yes"].shape

(3221, 4)

### Copy subset

In [19]:
PART_TO_ANALYZE = "Paragolpe Delantero"

metadata = common.load_metadata_dataframe('state.json')
metadata = metadata[metadata.apply(one_part_dataset.is_useful, axis=1)]

complaint_parts = pd.read_csv('../preprocessing/piezas_normalizadas.csv')
display(metadata.head())
display(complaint_parts.head())

,image,useful,angle
0,M204933/M204933001.jpg,yes,frente_acomp
1,M204933/M204933002.jpg,yes,frente_cond
2,M204933/M204933003.jpg,yes,atras_cond
3,M204933/M204933004.jpg,yes,atras_acomp
4,M204933/M204933005.jpg,yes,lado_acomp


,Siniestro,DENUNCIA,Pieza,Tarea,Horas,pieza_normalizada
0,577/2020,M245982,tapa baul hatch,Sustituir,2.50,Tapa de Baul
1,577/2020,M245982,luneta tras,SYC,1.00,Otro
2,577/2020,M245982,kit pegado,Sustituir,0.00,Otro
3,577/2020,M245982,paragolpe trasero,Sustituir,1.00,Paragolpe Trasero
4,577/2020,M245982,farol trasero izq,Sustituir,0.25,Farol Trasero Izquierdo


In [20]:
metadata.shape

(1665, 3)

In [21]:
def parts_info_from_complaint(complaint):
        df_complaint = complaint_parts[complaint_parts["DENUNCIA"] == complaint]
        return list(df_complaint[["pieza_normalizada", "Tarea", "Horas"]].to_records(index=False))

def get_part_category(row):
        is_visible = PART_TO_ANALYZE in common.angulo_pieza[row["angle"]]
        parts_info = parts_info_from_complaint(row["image"].split("/")[0])
        
        broken_part_info = None
        is_broken = False
        for (part, tarea, horas) in parts_info:
            if PART_TO_ANALYZE == part:
                is_broken = True
                broken_part_info = (tarea, horas)

        if is_visible:
            if is_broken:
                return ("roto", broken_part_info[0], float(broken_part_info[1]))
            else:
                return ("sano", "nada", 0)
        else:
            return ("no_visible", "nada", 0)
        
def generate_image_info(row):
    (estado, tarea, horas) = get_part_category(row)
    return (row["image"], estado, tarea, horas)

In [22]:
samples = metadata.apply(generate_image_info, axis=1).tolist()
samples_df = pd.DataFrame(samples)
samples_df.columns = ["image", "estado", "tarea", "horas"]

In [23]:
samples_df.head()

,image,estado,tarea,horas
0,M204933/M204933001.jpg,sano,nada,0.0
1,M204933/M204933002.jpg,sano,nada,0.0
2,M204933/M204933003.jpg,no_visible,nada,0.0
3,M204933/M204933004.jpg,no_visible,nada,0.0
4,M204933/M204933005.jpg,no_visible,nada,0.0


In [25]:
samples_df = samples_df[samples_df["estado"] == "roto"]

In [30]:
samples_df.shape

(424, 4)

In [28]:
FROM_PREFIX = '/Users/fcuevas/Documents/imgs/{}'
TO_PREFIX = "./imgs_to_classify/{}"

def copy_img(row):
    from_dir = FROM_PREFIX.format(row["image"])
    to_dir = TO_PREFIX.format(row["image"])
    
    os.makedirs(os.path.dirname(to_dir), exist_ok=True)
    copyfile(from_dir, to_dir)

samples_df.apply(copy_img, axis=1)

14      None
15      None
20      None
21      None
22      None
        ... 
1656    None
1657    None
1658    None
1661    None
1663    None
Length: 424, dtype: object